# VAL AND MOM IN THE SAME PLACE (Work-in-Progress)

[@sparshsah](https://github.com/sparshsah)

## setup

In [ ]:
from typing import Final

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from foggy_statslib import core as fc

In [ ]:
def _get_wave(
    a: float = 1,
    period: float = 4,
    sz: int = 256,
) -> pd.Series:
    b = 2 * np.pi / period
    domain = pd.Series(range(sz))
    wave = a * pd.Series(np.sin(b * domain))
    return wave


def _make_std_tsmom_sig(
    x: pd.Series,
    front_end: int = 0,
    back_end: int = 65,
) -> pd.Series:
    wdw = back_end - front_end
    sig = x.rolling(window=wdw).sum()
    # catch up
    sig = sig.shift(front_end)
    # standardize
    sig = sig / wdw**0.5
    return sig


def _get_one_corr(r: pd.Series, hzn: int = 1) -> float:
    """TSMOM evaluation e.g. Today's return vs Last month's return."""
    r_cum_lagged = r.rolling(window=hzn).sum().shift()
    return r.corr(r_cum_lagged)


def _get_all_corrs(r: pd.Series) -> pd.Series:
    corrs = pd.Series(
        {
            hzn:
            _get_one_corr(r=r, hzn=hzn)
            for hzn in range(1, 200)
        }
    ).rename_axis(index="hzn")
    return corrs

## Empirical Background: Value and Momentum Everywhere

["Value and Momentum Everywhere" (Asness-Moskowitz-Pedersen 2013)](https://www.aqr.com/Insights/Research/Journal-Article/Value-and-Momentum-Everywhere) was a seminal contribution to liquid-alternative relative-value factor investing, in both single-name and macro securities.

The following cross-sectional pricing anomalies (based on backward-looking observed returns) tend to stand up across geographies (e.g. emerging vs G10 vs currencies), asset classes (e.g. commodities vs government bonds), and time:
* Sign of spread return over `[0, -1 month]` reverses
* Sign of spread return over `[-2 months, -12 months]` continues
* Sign of spread return over `[-2 years, -5 years]` reverses

(Cliff himself often chides asset allocators for being "momentum investors at a value horizon", i.e. pulling money from underperforming managers and plowing it into outperforming managers based on trailing 3-year returns.)

Separately, the following directional (timeseries) pricing anomalies:
* Sign of own return over `[0, -1 week]` reverses
* Sign of own return over `[-2 weeks, -52 weeks]` continues
* Sign of own return over `[-2 years, -5 years]` reverses

## How is This Possible?

It initially blew my mind that these effects could coexist. Short-term returns reverse, but medium-term returns continue... isn't the medium term made up of just a bunch of short terms?? But here's an example:

`// TODO(sparshsah): I might have to just do it (ugh) as an AR process... :(`

In [140]:
np.random.seed(42)
r = pd.Series(
    np.random.normal(
        scale=0.01,
        size=T,
    )
)
# this should have 16% vol
r.std() * 261**0.5

0.16170130634343932

In [142]:
dose_of_val = -r.rolling(21  - 0 ).sum().shift(0 ).shift()  / (21  - 0 )**0.5
dose_of_mom =  r.rolling(261 - 21).sum().shift(21).shift()  / (261 - 21)**0.5

In [143]:
dose_of_val.std()*261**0.5, dose_of_mom.std()*261**0.5

(0.16348454891083058, 0.16166577651086284)

In [144]:
dose_of_val.corr(dose_of_mom)

0.008385607691348787

In [158]:
x = (dose_of_val + dose_of_mom) / 2**0.5
x.std() * 261**0.5

0.16327831120450118

In [159]:
x.corr(dose_of_val)

0.7140459991600808

In [160]:
x.corr(dose_of_mom)

0.706061880931298

In [165]:
# with r
x.corr(
    r.rolling(21).sum().shift()
)

-0.7140459991600819

In [164]:
x.corr(
    r.rolling(261).sum().shift()
)

0.47284131561403275

In [166]:
x.corr(
    r.rolling(261 - 21).sum().shift(21).shift()
)

0.7060618809312974

In [168]:
# with itself
x.corr(
    x.rolling(21).sum().shift()
)

0.7259139766628293

In [167]:
x.corr(
    x.rolling(261).sum().shift()
)

0.2272290989244234

In [169]:
x.corr(
    x.rolling(261 - 21).sum().shift(21).shift()
)

0.12193145569903413

## Theoretical Question: What's the Story?

I propose a behavioral story that is (a) self-consistent and (b) at least first-order consistent with stylized facts about market microstructure that could give rise to this effect.